In [ ]:
import os
import re
import sys
import numpy as np
from scipy.io import savemat
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

if not '..' in sys.path:
    sys.path.append('..')
from pfcommon import parse_Amat_file, parse_vars_file

matplotlib.rc('font', **{'family': 'sans-serif', 'sans-serif': 'Arial', 'size': 9})
matplotlib.rc('axes', **{'linewidth': 0.75})

%matplotlib inline

In [ ]:
base_folder = 'C:\\Users\\Terna_Inerzia\\Desktop\\modal_analysis\\IEEE39'
sys_name = 'IEEE39'
conditions = 'default', 'G02_double_H'
fmin, fmax, steps_per_decade = -6, 2, 1000
F, TF, gen_names = {}, {}, {}
Htot, Etot, Mtot = {}, {}, {}
H, S = {}, {}
P, Q = {}, {}
A, cols, var_names, model_names = {}, {}, {}, {}
mag, phase = {}, {}
B = {}
for cond in conditions:
    data_file = os.path.join(base_folder, cond, 'TF_{}_{}_{}_{}.npz'.
                             format(sys_name, fmin, fmax, steps_per_decade))
    A_file = os.path.join(base_folder, cond, 'Amat.mtl')
    vars_file = os.path.join(base_folder, cond, 'VariableToIdx_Amat.txt')
    A[cond] = parse_Amat_file(A_file)
    cols[cond],var_names[cond],model_names[cond] = parse_vars_file(vars_file)
    data = np.load(data_file, allow_pickle=True)
    F[cond] = data['F']
    gen_names[cond] = data['gen_names']
    Htot[cond] = data['inertia']
    Etot[cond] = data['energy']
    Mtot[cond] = data['momentum']
    H_dict, S_dict = data['H'].item(), data['S'].item()
    H[cond] = np.array([H_dict[sm] for sm in gen_names[cond]])
    S[cond] = np.array([S_dict[sm] for sm in gen_names[cond]])
    PF = data['PF_without_slack'].item()
    n_SMs = len(gen_names[cond])
    P[cond], Q[cond] = np.zeros(n_SMs), np.zeros(n_SMs)
    for i,sm in enumerate(gen_names[cond]):
        if sm in PF['SMs']:
            key = sm
        else:
            key = sm + '____GEN_____'
        P[cond][i] = PF['SMs'][key]['P']
        Q[cond][i] = PF['SMs'][key]['Q']
    B[cond] = H[conditions[0]] / H[cond]

In [ ]:
Nf = int(fmax - fmin) * steps_per_decade + 1
F = np.logspace(fmin, fmax, Nf)
J = A[cond]
Nv = J.shape[0]
I = np.eye(Nv)
M = np.zeros((Nf, Nv, Nv), dtype=complex)

In [ ]:
TF = {cond: np.zeros((Nf,Nv), dtype=complex) for cond in conditions}
jdx, = np.where(B['G02_double_H'] != 1)
for cond in conditions:
    b = np.zeros(Nv)
    omega_col_idx, = np.where([vname == 'speed' and 'ElmSym' in mname \
                               for vname, mname in zip(var_names[cond],model_names[cond])])
#     b[omega_col_idx[jdx]] = B[cond][jdx]
    b[omega_col_idx] = B[cond]
    for i in range(Nf):
        M[i,:,:] = np.linalg.inv(-J + 1j*2*np.pi*F[i]*I)
        TF[cond][i,:] = M[i,:,:] @ b
    TF[cond] = TF[cond][:,omega_col_idx]
    mag[cond] = 20*np.log10(np.abs(TF[cond]))
    phase[cond] = np.angle(TF[cond])

In [ ]:
outfile = 'TF_{}_{}_{}_{}'.format(sys_name, fmin, fmax, steps_per_decade)
mdict = {'conditions': conditions,
         'H': np.array(list(Htot.values())),
         'M': np.array(list(Mtot.values())),
         'E': np.array(list(Etot.values()))}
for cond in conditions:
    mdict['F_'+cond] = F
    mdict['TF_'+cond] = TF[cond]
    mdict['mag_'+cond] = mag[cond]
    mdict['phase_'+cond] = phase[cond]
    mdict['gen_names_'+cond] = gen_names[cond]
    mdict['H_'+cond] = H[cond]
    mdict['S_'+cond] = S[cond]
    mdict['P_'+cond] = P[cond]
    mdict['Q_'+cond] = Q[cond]
savemat(os.path.join(base_folder, outfile + '.mat'), mdict)

In [ ]:
%matplotlib notebook
fig,ax = plt.subplots(2, 1, figsize=(6,5), sharex=True)
col = 'krgb'
for i,cond in enumerate(conditions):
    ax[0].plot(F, mag[cond][:,0], col[i], lw=1, label=cond.replace('_',' ') + \
               r': M={:.0f} MW$\cdot \mathrm{{s}}^2$'.format(Mtot[cond]))
    ax[0].plot(F, mag[cond], col[i], lw=1)
#     ax[1].plot(F, phase[cond], col[i], lw=1)
    ϕ = phase[cond]
    ϕ[ϕ>1] -= 2*np.pi
    ax[1].plot(F, ϕ, col[i], lw=1)
ax[0].legend(loc='lower left', frameon=False, fontsize=10)
ax[0].set_xscale('log')
ax[1].set_xlabel('Frequency [Hz]')
ax[0].set_ylabel('Magnitude [dB20]')
ax[1].set_ylabel('Phase [rad]')
sns.despine()
fig.tight_layout()
plt.savefig(os.path.join(base_folder, outfile + '.pdf'))